In [ ]:
# Atualização das bibliotecas
!pip install --upgrade keras tensorflow

In [ ]:
# Download do dataset de Treinamento
!gdown https://drive.google.com/uc?id=1xQqTSUA-celn_KRB9Qpx77g54KR0WHaH
!unzip Leishmania_Treinamento.zip
!rm Leishmania_Treinamento.zip

In [1]:
# Fixando seeds pra ter resultados mais consistentes
import numpy as np
import tensorflow as tf
import random as python_random

np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

In [2]:
# Imports
from os import listdir
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import os

In [3]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# Inicialização do modelo
classificador = Sequential()

tamanho_imagem = (256, 256)
# Carregamento da arquitetura
transfer = ResNet152V2(include_top = False, input_shape = (tamanho_imagem[0], 
                                                           tamanho_imagem[1], 3))
for camada in transfer.layers:
  camada.trainable = True

classificador.add(transfer)

# Saída do modelo
classificador.add(Flatten())
classificador.add(Dense(1, activation = 'sigmoid'))

classificador.compile(optimizer = Adam(learning_rate = 0.00001),
               loss = 'binary_crossentropy', metrics = ['accuracy'])

batch = 10

# Nenhuma forma de Augmentation
# Rescale é apenas normalização
datagen = ImageDataGenerator(rescale=1./255)


grupo_treinamento = datagen.flow_from_directory('Leishmania_Training_Dataset/treinamento', 
                                                target_size=tamanho_imagem, 
                                                batch_size=batch, 
                                                class_mode='binary')

grupo_validacao = datagen.flow_from_directory('Leishmania_Training_Dataset/validacao', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')

classificador.fit(grupo_treinamento, epochs=10, validation_data=grupo_validacao)

Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/10
40/40 [==============================] - 19s 474ms/step - loss: 0.5683 - accuracy: 0.7200 - val_loss: 0.6131 - val_accuracy: 0.6600
Epoch 2/10
40/40 [==============================] - 16s 409ms/step - loss: 0.1048 - accuracy: 0.9700 - val_loss: 0.4567 - val_accuracy: 0.8000
Epoch 3/10
40/40 [==============================] - 17s 413ms/step - loss: 0.0565 - accuracy: 0.9850 - val_loss: 0.4281 - val_accuracy: 0.8200
Epoch 4/10
40/40 [==============================] - 16s 410ms/step - loss: 0.0262 - accuracy: 0.9975 - val_loss: 0.3454 - val_accuracy: 0.8700
Epoch 5/10
40/40 [==============================] - 17s 416ms/step - loss: 0.0286 - accuracy: 0.9975 - val_loss: 0.3206 - val_accuracy: 0.8700
Epoch 6/10
40/40 [==============================] - 16s 411ms/step - loss: 0.0170 - accuracy: 1.0000 - val_loss: 0.3235 - val_accuracy: 0.8800
Epoch 7/10
40/40 [==============================] - 16s 412m

In [ ]:
# Download do dataset de Teste
!gdown https://drive.google.com/uc?id=1tTBldPLMQk8jLLDjBaxwbl_FXpYNS0Ux
!unzip Leishmania_Teste.zip
!rm Leishmania_Teste.zip

In [4]:
# Funções

# Preprocessamento das imagens
def carregar_imagem(arquivo):
  img = image.load_img(arquivo, target_size=tamanho_imagem)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = img/255

  return img

# Carregar imagens em uma lista
def predicao_teste(pasta_teste):
  imagens = []
  for pasta in listdir(pasta_teste):
    for arquivo in listdir(pasta_teste + '/' + pasta):
      imagens.append(carregar_imagem(pasta_teste + pasta + '/' + arquivo))
  
  return imagens

In [5]:
imagens = np.vstack(predicao_teste('Leishmania_Test_Dataset/'))
respostas_modelo = classificador.predict_classes(imagens)
respostas_teste = datagen.flow_from_directory('Leishmania_Test_Dateset/', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')
print('Respostas do modelo\n' + str(np.concatenate(respostas_modelo)))
print('Respostas reais\n' + str(respostas_teste.classes))

# Matriz de confusão
matriz_de_confusao = confusion_matrix(respostas_teste.classes, respostas_modelo)

# Metricas
# Marvin pediu especificamente pra gente avaliar os valores de sensibilidade e
# especificidade
TrueNegative, FalsePositive, FalseNegative, TruePositive = matriz_de_confusao.ravel()

# Precisão
print('Precisao ' + str((TruePositive + TrueNegative) / len(respostas_modelo)))

# Sensibilidade
print('Sensibilidade ' + str(TruePositive / (TruePositive + FalseNegative)))

# Especificidade
print('Especificidade ' + str(TrueNegative / (FalsePositive + TrueNegative)))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


FileNotFoundError: [Errno 2] No such file or directory: 'Leishmania_Test_Dateset/'

In [ ]:
# Salvar modelo treinado em um arquivo
classificador.save('modelo_leishmania.h5')